# Librairies à importer

In [ ]:
import pandas as pd
import ydata_profiling
from ydata_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler

# Étape 1 : Préparation des données

## Néttoyage de la base

In [2]:
data = pd.read_csv('base_excel/songs_with_release_year.csv')

print(data.info())
print(data.describe())

data['decenie'] = (data['release_year'] // 10) *10

### Compter les fréquences par décennie
frequency_by_decade = data['decenie'].value_counts().sort_index()

### Afficher les résultats
print(frequency_by_decade)
#data = data[data['decenie'] >1950]

In [3]:
## suppression des lignes 
data = data[data['release_year'] != 0] #où l'années est inférieur à 1960
data = data[(data['duration_ms'] >= 73897.375) & (data['duration_ms'] <= 361032.375)]
data = data[(data['liveness'] <= 0.8)] # Une valeur supérieure à 0,8 indique une forte probabilité que le morceau soit joué en direct.
data = data[(data['instrumentalness'] <= 0.5)] # Une valeur supérieure à 0,5 indique une forte probabilité que le morceau soit un instrumentale
data = data[(data['speechiness'] <= 0.66)] # Une valeur supérieure à 0,66 indique une forte probabilité que le morceau soit  constituées entièrement de mots parlés (talk-show, livre audio, poésie)



In [4]:
## Suppression des variables
variables_a_supprimer = ['pourcentage_langue3', 'langue3', 'pourcentage_langue2', 'langue2', 'langue1', 'pourcentage_langue1', 'album_name', 'artists', 'name', 'lyrics']
data = data.drop(columns=variables_a_supprimer, errors='ignore')

obtenir_modalite=['key','mode']
for var in obtenir_modalite:
    modalites_uniques = data[var].unique()
    print(f'les modalité de la variable {var} sont {modalites_uniques}')


frequence_key = data['key'].value_counts()
frequence_mode = data['mode'].value_counts()
print(frequence_key)
print(frequence_mode)

In [5]:
# Dictionnaires de mapping
note_to_int = {
    'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3,
    'E': 4, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 'Ab': 8,
    'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
}

mode_to_int = {
    'Minor': 0, 'Major': 1
}

def convert_key(value, mapping):
    try:
        # Convertir les valeurs décimales en entiers
        if isinstance(value, str) and '.' in value:
            value = int(float(value))
        # Convertir les notes musicales en entiers
        return int(mapping.get(value, value))  # Retourner la valeur originale si elle n'est pas trouvée dans le dictionnaire
    except ValueError:
        # Si la conversion échoue, retourner la valeur originale
        return value

# Appliquer la conversion
data['key'] = data['key'].apply(lambda x: convert_key(x, note_to_int))
data['mode'] = data['mode'].apply(lambda x: convert_key(x, mode_to_int))

obtenir_modalite=['key','mode']
for var in obtenir_modalite:
    modalites_uniques = data[var].unique()
    print(f'les modalité de la variable {var} sont {modalites_uniques}')

frequence_key = data['key'].value_counts()
frequence_mode = data['mode'].value_counts()
print(frequence_key)
print(frequence_mode)

## Exploration de la data

### Rapport pandas profiling

In [ ]:
profile = ProfileReport(data, title="Pandas Profiling Report")
profile.to_file("rapport.html")

dd=data['release_year'].value_counts().sort_values()
print(dd)

### Statistiques descriptives

#### Fonction pour identifier les outliers selon la méthode IQR


In [7]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    is_outlier = (df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))
    return is_outlier

#### Fonction pour compter le nombre d'outliers par individu

In [8]:
def count_outliers(df, variables_to_check):
    # Initialiser une série qui va compter les outliers pour chaque individu
    outliers_count = pd.Series(0, index=df.index)
    
    # Parcourir les colonnes et appliquer la fonction des outliers
    for var in variables_to_check:
        outliers_count += detect_outliers_iqr(df, var).astype(int)
    
    # Retourner le nombre d'outliers par individu
    return outliers_count

#### Fonction pour générer les statistiques descriptives (avec nombre d'outliers par variable)


In [9]:
def analyze_numerical_vars(df, numeric_cols):
    ##### Initialiser un DataFrame pour stocker les statistiques
    stats = pd.DataFrame(index=numeric_cols, columns=[
        'Moyenne', 'Médiane', 'Variance', 'Type de Distribution', 'Minimum', 'Maximum', 'Asymétrie', 'Nombre d\'Outliers'
    ])
    
    for col in numeric_cols:
        # Calcul des statistiques de base
        stats.loc[col, 'Moyenne'] = df[col].mean()
        stats.loc[col, 'Médiane'] = df[col].median()
        stats.loc[col, 'Variance'] = df[col].var()
        stats.loc[col, 'Type de Distribution'] = 'Normale' if np.abs(df[col].skew()) < 0.5 else 'Non Normale'
        stats.loc[col, 'Minimum'] = df[col].min()
        stats.loc[col, 'Maximum'] = df[col].max()
        stats.loc[col, 'Asymétrie'] = df[col].skew()
        
        # Détection des outliers via la fonction IQR
        is_outlier = detect_outliers_iqr(df, col)
        
        # Stocker le nombre d'outliers pour chaque variable
        stats.loc[col, 'Nombre d\'Outliers'] = is_outlier.sum()

    return stats

#### Fonction pour afficher le tableau de fréquence

In [10]:
def frequency_table(df, col):
    col_values = df[col]
    # Créer des intervalles
    bins = np.linspace(col_values.min(), col_values.max(), 51)  # 50 intervalles
    freq, edges = np.histogram(col_values, bins=bins)
    
    # Créer un DataFrame pour la fréquence
    freq_table = pd.DataFrame({
        'Intervalle': [f'({edges[i]:.2f}, {edges[i+1]:.2f}]' for i in range(len(edges)-1)],
        'Fréquence': freq
    })
    return freq_table

In [ ]:
variables_to_use1 = data.select_dtypes(include=[np.number]).columns.tolist()

#1. Calculer les statistiques
stats_table = analyze_numerical_vars(data, variables_to_use1)
print(stats_table)

# 2. Calculer le nombre d'outliers par individu
data['outliers_count'] = count_outliers(data, variables_to_use1)

In [12]:
data = data[(data['outliers_count'] < 2)] # Une valeur supérieure à 0,8 indique une forte probabilité que le morceau soit joué en direct.


In [ ]:
variables_to_use1 = data.select_dtypes(include=[np.number]).columns.tolist()

# Calculer les statistiques
stats_table2 = analyze_numerical_vars(data, variables_to_use1)
print(stats_table2)

In [ ]:


for col in data.select_dtypes(include=[np.number]).columns:
    print(f"\nTableau de fréquence pour {col}:")
    print(frequency_table(data, col))

## Transormation

### encodage cyclique pour la varibale keys

In [ ]:
def encode_cyclic(value, N):
    # N est le nombre de valeurs uniques (12 pour les notes musicales)
    radians = 2 * np.pi * value / N
    return np.sin(radians), np.cos(radians)

# Appliquer la fonction à la colonne 'key'
data['key_sin'], data['key_cos'] = zip(*data['key'].apply(lambda x: encode_cyclic(x, 12)))

### Winsorisation de loudness

In [16]:
def winsorize_variable(data, column_name, mode='both'):
    # Vérifier si la colonne existe
    if column_name not in data.columns:
        raise ValueError(f"La colonne '{column_name}' n'existe pas dans le DataFrame.")
    
    # Calculer les quartiles
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    
    # Calculer l'IQR (écart interquartile)
    IQR = Q3 - Q1
    
    # Calculer les limites pour détecter les valeurs extrêmes
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Appliquer la Winsorisation manuelle en fonction du mode
    winsorized_column_name = f"{column_name}_winsorized"
    
    if mode == 'both':
        data[winsorized_column_name] = np.clip(data[column_name], lower_bound, upper_bound)
    elif mode == 'upper':
        data[winsorized_column_name] = np.clip(data[column_name], -np.inf, upper_bound)
    elif mode == 'lower':
        data[winsorized_column_name] = np.clip(data[column_name], lower_bound, np.inf)
    else:
        raise ValueError("Le mode doit être 'both', 'lower' ou 'upper'.")
    
    # Afficher les statistiques avant et après la Winsorisation
    variable_afficher = [column_name, winsorized_column_name]
    print(f"Avant et après Winsorisation pour '{column_name}':\n", analyze_numerical_vars(data, variable_afficher))
    
    print(f"Limites : Supérieure = {upper_bound}, Inférieure = {lower_bound}")
    
    return data


In [ ]:
data = winsorize_variable(data, 'loudness', mode='both')


data = winsorize_variable(data, 'tempo', mode='both')

data = winsorize_variable(data, 'duration_ms', mode='both')

### Transformation logarithmique

In [ ]:
data['speechiness_log'] = np.log(data['speechiness'] + 1e-6)

data['speechiness_transformed'] = np.log(data['speechiness'] + 1)
j'ai essayer ceci mais cela n'a pas réduit le nombre d'outlier

In [ ]:
data['instrumentalness_log'] = np.log(data['instrumentalness'] + 1e-6)

In [ ]:
data['liveness_log'] = np.log(data['liveness'] + 1e-6)

In [ ]:
data['duration_log'] = np.log(data['duration_ms'])  

In [ ]:
#analyse descriptives des variables transformer
varr=['speechiness_log','instrumentalness_log','liveness_log', 'duration_log']
print(analyze_numerical_vars(data, varr))

### Z-score Normalization (StandardScaler) avec scikit-learn :
 loudness_winsorized

In [22]:
# Fonction pour normaliser les colonnes numériques et afficher les statistiques
def normalize_zscore(df, numeric_cols):
    """
    Cette fonction applique la normalisation Z-score aux colonnes numériques spécifiées
    et affiche les statistiques avant et après la normalisation.
    
    Paramètres:
    - df: DataFrame contenant les données
    - numeric_cols: Liste des colonnes numériques à normaliser
    
    Retour:
    - df_scaled: DataFrame avec les colonnes normalisées
    - stats_before: DataFrame des statistiques avant normalisation
    - stats_after: DataFrame des statistiques après normalisation
    """
    
    # Initialiser le scaler Z-score
    scaler = StandardScaler()
    
    # Afficher les statistiques avant normalisation
    print("Statistiques avant normalisation :")
    stats_before = analyze_numerical_vars(df, numeric_cols)
    print(stats_before)
    
    # Appliquer la normalisation Z-score aux colonnes numériques
    df_scaled = df.copy()
    df_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    
    # Afficher les statistiques après normalisation
    print("\nStatistiques après normalisation :")
    stats_after = analyze_numerical_vars(df_scaled, numeric_cols)
    print(stats_after)
    
    return df_scaled, stats_before, stats_after



In [ ]:
# Sélectionner les colonnes numériques que tu veux normaliser
numeric_cols = ['loudness_winsorized', 'speechiness_log', 'instrumentalness_log', 'liveness_log']

# Appliquer la normalisation Z-score et afficher les statistiques avant et après
data_scaled_zscore, stats_before, stats_after = normalize_zscore(data, numeric_cols)


### suppression des variables originales après transformation

In [91]:
data_scaled_zscore = data_scaled_zscore.drop(data_scaled_zscore['loudness'], errors='ignore')

In [ ]:
data_scaled_zscore.describe()

In [ ]:
def stat_des(colo):
    # Initialiser un DataFrame pour stocker les statistiques
    stats = pd.DataFrame(columns=[
        'Moyenne', 'Médiane', 'Variance', 'Type de Distribution', 'Minimum', 'Maximum', 'Asymétrie', 'Outliers'
    ])
    
    # Calcul des statistiques basiques
    stats.loc['Valeurs', 'Moyenne'] = colo.mean()
    stats.loc['Valeurs', 'Médiane'] = colo.median()
    stats.loc['Valeurs', 'Variance'] = colo.var()
    stats.loc['Valeurs', 'Type de Distribution'] = 'Normale' if np.abs(colo.skew()) < 0.5 else 'Non Normale'
    stats.loc['Valeurs', 'Minimum'] = colo.min()
    stats.loc['Valeurs', 'Maximum'] = colo.max()
    stats.loc['Valeurs', 'Asymétrie'] = colo.skew()
    
    # Détection des valeurs extrêmes avec l'IQR
    Q1 = colo.quantile(0.25)
    Q3 = colo.quantile(0.75)
    IQR = Q3 - Q1
    
    # Nombre de valeurs extrêmes (outliers)
    outliers = ((colo < (Q1 - 1.5 * IQR)) | (colo > (Q3 + 1.5 * IQR))).sum()
    
    # Stocker le nombre de valeurs extrêmes dans le tableau des statistiques
    stats.loc['Valeurs', 'Outliers'] = outliers
    
    return stats

# Appliquer la fonction sur la colonne 'loudness'
loudness_apres = stat_des(data['loudness_winsorized'])
print(loudness_apres)
